# add metadata to dataset
Original DPK repo does not save the video file path and frame number for frames. I need to find the videos and frame nums for images already in the training set by looping over ALL VIDS AND ALL IMAGES to find matches. Head free videos only atm!

In [23]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from shutil import copyfile
import numpy as np
import h5py
import yaml
import cv2
import os
root_dir = r'D:\github\fish-tracking'  # github repo

# settings
dataset_name = r'D:\github\fish-tracking\label\datasets\tank_dataset_10.h5'
new_dataset_name = r'D:\github\fish-tracking\label\datasets\tank_dataset_11.h5'

# load dataset
with h5py.File(dataset_name, 'r') as f:
    dataset = {key: value[...] for key, value in f.items()}
imgs = dataset['images'][...,0]  # frame num x height x width
n_imgs = imgs.shape[0]

# load config
with open(os.path.join(root_dir, 'config.yaml'), 'r') as file:
    cfg = yaml.safe_load(file)['free']

def update_dataset(old_dset, new_dset, vid_names, frame_nums):
    ''' copy old_dset to new_dset and add video_name and frame_number fields '''
    copyfile(dataset_name, new_dataset_name)
    with h5py.File(new_dataset_name, 'r+') as f:
        
        f.create_dataset(
            "video_name",
            frame_nums.shape,
            dtype="S64",
            data=np.array(vid_names, dtype='S64')
        )

        f.create_dataset(
            "frame_number",
            frame_nums.shape,
            dtype=np.int,
            data=frame_nums
        )

# be careful!
The following code will overwrite `new_dataset` if it already exists!

In [6]:
all_vid_names = [os.path.join(f.path, 'concatenated.avi') for f in os.scandir(cfg['data_dir'])]
frame_nums = np.full(n_imgs, -1, dtype=np.int)  # store frame numbers when then are found
vid_names = ['unknown']*n_imgs
match_row = 192  # only check that this row matches, otherwise way way way way too slow!

# tests
# 20201210	Greg	1224
# 20201210	Neil	12756, 25207, 27834, 81059
# all_vid_names = [r'Z:\C1\free\vids\20201210_Greg\concatenated.avi', 
#                  r'Z:\C1\free\vids\20201210_Neil\concatenated.avi']


for vid_name in all_vid_names:
    if os.path.exists(vid_name):

        # load video
        vid = cv2.VideoCapture(vid_name)
        n_frames_approx = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
        print('Checking {} for matches...'.format(vid_name))

        for frame_num in tqdm(range(round(n_frames_approx*1.01))):

            # read frame
            got_frame, frame = vid.read()
            if not got_frame:
                break

            # check for matching rows
            match_idx = np.where(np.all(imgs[:,match_row] == frame[match_row,:,0], axis=1))[0]
            
            # now check entire frame
            if len(match_idx)>0:
                match_idx = np.where(np.all(imgs == frame[...,0], axis=(1,2)))[0]
            
            # save if match is found
            if len(match_idx)>0:
                match_idx = match_idx[0]
                frame_nums[match_idx] = frame_num
                vid_names[match_idx] = vid_name
                print('  Found a match! Dataset image {} is frame {} of {} '.
                      format(match_idx, frame_num, vid_name))
                update_dataset(dataset_name, new_dataset_name,
                               vid_names, frame_nums)
                

Checking Z:\C1\free\vids\20200903\concatenated.avi for matches...



Checking Z:\C1\free\vids\20200907\concatenated.avi for matches...



Checking Z:\C1\free\vids\20200909\concatenated.avi for matches...



Checking Z:\C1\free\vids\20200911\concatenated.avi for matches...



Checking Z:\C1\free\vids\20200913\concatenated.avi for matches...


...


Checking Z:\C1\free\vids\Fish1\concatenated.avi for matches...



Checking Z:\C1\free\vids\Fish2\concatenated.avi for matches...



Checking Z:\C1\free\vids\Fish3\concatenated.avi for matches...



Checking Z:\C1\free\vids\Fish4\concatenated.avi for matches...



Checking Z:\C1\free\vids\Fish5\concatenated.avi for matches...


In [25]:
# check new data
with h5py.File(new_dataset_name, 'r') as f:
    temp = {key: value[...] for key, value in f.items()}

for n, f in zip(temp['frame_number'], temp['video_name']):
    print('frame {}, video {}'.format(n, f))

frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
f

In [24]:
new_dataset_name


'D:\\github\\fish-tracking\\label\\datasets\\tank_dataset_11.h5'